<a href="https://colab.research.google.com/github/Logeshwaran32/Singapore-resale/blob/main/Singapore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#!pip install numpy
#!pip install pandas
#!pip install matplotlib
#!pip install scikit-learn
!pip install streamlit
!pip install pyngrok

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore
import statsmodels.api as sm
from wordcloud import WordCloud
import plotly.express as px
import joblib
import matplotlib.patches as mpatches
import plotly.graph_objects as go
import calendar
import streamlit as st

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score
from math import sqrt


1.   Data Preprocessing and cleaning
2.   Reading all the datasets for a basic understanding



In [ ]:
# Read the csv file into dataframe
df1 = pd.read_csv("/content/drive/MyDrive/DataSet/Singapore_resale_flat_prices_01.csv")

# Print the basic information and first few rows of the dataframe
print(df1.info())
print(df1.head())

In [ ]:
# Read the csv file into dataframe
df2 = pd.read_csv("/content/drive/MyDrive/DataSet/Singapore_resale_flat_prices_02.csv")

# Print the basic information and first few rows of the dataframe
print(df2.info())
print(df2.head())

In [ ]:
# Read the csv file into dataframe
df3 = pd.read_csv("/content/drive/MyDrive/DataSet/Singapore_resale_flat_prices_03.csv")

# Print the basic information and first few rows of the dataframe
print(df3.info())
print(df3.head())

In [ ]:
# Read the csv file into dataframe
df4 = pd.read_csv("/content/drive/MyDrive/DataSet/Singapore_resale_flat_prices_04.csv")

# Print the basic information and first few rows of the dataframe
print(df4.info())
print(df4.head())

In [ ]:
# Read the csv file into dataframe
df5 = pd.read_csv("/content/drive/MyDrive/DataSet/Singapore_resale_flat_prices_05.csv")

# Print the basic information and first few rows of the dataframe
print(df5.info())
print(df5.head())

Splitting the numerical part of the remaining lease from DF5

In [ ]:
# Extract the numeric part of 'remaining_lease', split by space, and convert to integer
df5['remaining_lease'] = df5['remaining_lease'].str.split(' ').str[0].astype('int64')

# Display the updated 'remaining_lease' column
print(df5['remaining_lease'])

# Print the basic information and first few rows of the dataframe
print(df5.info())
print(df5.head())

Merging all the datasets

In [ ]:
# Merging datasets df1, df2, df3, df4 and df5
df_combined = pd.concat([df1, df2, df3, df4, df5], ignore_index=True)

# Print the basic information and first few rows of the merged dataframe
print(df_combined.info())
print(df_combined.head())

Splitting the month column to month and year

In [ ]:
# Split 'month' into 'year' and 'month' columns
df_combined[['year', 'month']] = df_combined['month'].str.split('-', expand=True)

# Convert 'year' and 'month' columns to numeric types if needed
df_combined['year'] = pd.to_numeric(df_combined['year'])
df_combined['month'] = pd.to_numeric(df_combined['month'])

# Display the basic information and the updated DataFrame
print(df_combined.info())
print(df_combined.head())

Calculating the remaining lease for the Nan values

In [ ]:
# Calculate remaining lease for rows with NaN values
df_combined['remaining_lease'].fillna(99 - (df_combined['year'] - df_combined['lease_commence_date']), inplace=True)

# Verify the results
print(df_combined[['lease_commence_date', 'year', 'remaining_lease']])

Checking for repetitions in categorical columns

In [ ]:
# Printing unique values of the categorical columns to check for repetitions
print(df_combined['town'].unique())
print(df_combined['flat_type'].unique())
print(df_combined['block'].unique())
print(df_combined['street_name'].unique())
print(df_combined['storey_range'].unique())
print(df_combined['flat_model'].unique())

In [ ]:
# Changing the case to uppercase for consistency
df_combined['flat_model'] = df_combined['flat_model'].str.upper()

# Printing the unique values
print(df_combined['flat_model'].unique())

In [ ]:
# Convert 'flat_type' values to lowercase
df_combined['flat_type'] = df_combined['flat_type'].str.lower()

# Standardize values to a common set of categories
type_mapping = {
    '1 room': '1 room',
    '3 room': '3 room',
    '4 room': '4 room',
    '5 room': '5 room',
    '2 room': '2 room',
    'executive': 'executive',
    'multi generation': 'multi generation',
    'multi-generation': 'multi generation'
}

df_combined['flat_type'] = df_combined['flat_type'].map(type_mapping)

# Changing the case to Uppercase
df_combined['flat_type'] = df_combined['flat_type'].str.upper()

# Display the unique values
print(df_combined['flat_type'].unique())

Removing the outliers in the columns 'remaining_lease' and 'storey_range'

In [ ]:
# Checking for number of rows with remaining lease period more than 99
print((df_combined['remaining_lease']>99).sum())

# Create a boolean mask to identify rows containing any of the specified values
mask = df_combined['remaining_lease']>99

# Use the boolean mask to drop rows
df_combined = df_combined[~mask]

# Display the DataFrame after dropping rows
print(df_combined)

In [ ]:
# Drop rows where the 'storey_range' column contains specific values
values_to_drop = ['01 TO 05', '06 TO 10', '11 TO 15', '16 TO 20', '21 TO 25', '26 TO 30', '31 TO 35', '36 TO 40']

# Create a boolean mask to identify rows containing any of the specified values
mask = df_combined['storey_range'].isin(values_to_drop)

# Use the boolean mask to drop rows
df_combined = df_combined[~mask]

# Display the DataFrame after dropping rows
print(df_combined)

Splitting the Storey range to lower bound and upper bound

In [ ]:
# Split the 'storey_range' into lower and upper bounds
df_combined[['lower_bound', 'upper_bound']] = df_combined['storey_range'].str.split(' TO ', expand=True)

# Convert the lower and upper bounds to numeric values
df_combined['lower_bound'] = pd.to_numeric(df_combined['lower_bound'])
df_combined['upper_bound'] = pd.to_numeric(df_combined['upper_bound'])

# Display the DataFrame with lower and upper bounds
print(df_combined[['storey_range', 'lower_bound', 'upper_bound']])

In [ ]:
# Adding a new column 'price_per_sqm'
df_combined['price_per_sqm'] = df_combined['resale_price'] / df_combined['floor_area_sqm']

# Adding a new column 'years_holding'
df_combined['years_holding'] = df_combined['year'] - df_combined['lease_commence_date']

# Adding a new column 'current_remaining_lease'
df_combined['current_remaining_lease'] = df_combined['remaining_lease'] - (2024 - df_combined['year'])

# Display the DataFrame with the new column
print(df_combined[['resale_price', 'floor_area_sqm', 'price_per_sqm']])
print(df_combined[['year', 'lease_commence_date', 'remaining_lease', 'years_holding', 'current_remaining_lease']])

In [21]:
# Saving the clean dataset to a file
df_combined = df_combined.reset_index(drop=True)
df_combined.to_csv('/content/drive/MyDrive/DataSet/Singapore_resale_flat_prices_updated.csv', index=False)



*   Exploratory Data Analysis
*   Displaying basic statistics of the columns



In [ ]:
# Reading the dataset
df = pd.read_csv('/content/drive/MyDrive/DataSet/Singapore_resale_flat_prices_updated.csv')

# Display basic information about the dataset
print(df.info())

# Display summary statistics of numerical columns
print(df.describe())

# Display the first few rows of the dataset
print(df.head())

# Check for missing values
print(df.isnull().sum())

Univariate Analysis

In [ ]:
# Histogram
plt.figure(figsize=(12, 6))
sns.histplot(df['resale_price'], bins=20, kde=True)
plt.title('Histogram: Resale Price')
plt.xlabel('Resale Price')
plt.ylabel('Frequency')
plt.show()


# Box Plot
plt.figure(figsize=(12, 6))
sns.boxplot(x=df['month'])
plt.title('Box Plot: Month of Sale')
plt.xlabel('Month of Sale')
plt.show()

# Bar Plot
plt.figure(figsize=(12, 6))
sns.countplot(x='lease_commence_date', data=df)
plt.title('Bar Plot: Date of lease commencement')
plt.xlabel('Date of lease commencement')
plt.ylabel('Count')
plt.xticks(rotation=90)
plt.show()



In [ ]:
# Scatter Plot
sns.scatterplot(x='year', y='price_per_sqm', data=df)
plt.title('Scatter Plot: Price per square meter vs Year')
plt.show()

# Line plot
sns.lineplot(x='flat_model', y='price_per_sqm', data=df)
plt.title('Line Plot: Flat Model vs Price per square meter')
plt.xticks(rotation=90)
plt.show()

# Heatmap
sns.heatmap(df[['price_per_sqm', 'lease_commence_date']].corr(), annot=True, cmap='coolwarm')
plt.title('Heatmap: Correlation between Price per square meter and Lease commencement year')
plt.show()

# Bubble plot
sns.scatterplot(x='remaining_lease', y='price_per_sqm', size='floor_area_sqm', data=df)
plt.title('Bubble Plot: Price per square meter vs Remaining lease with floor_area_sqm as size')
plt.show()

Detect aberrant or missing values

In [ ]:
# Check for missing values
missing_values = df.isnull().sum()
print("Missing Values:")
print(missing_values)

# Detect outliers using Z-score
z_scores = zscore(df[['resale_price', 'floor_area_sqm']])
outliers = (abs(z_scores) > 3).all(axis=1)
print("Number of outliers:", outliers.sum())

In [ ]:
# Heatmap
sns.heatmap(df[['resale_price', 'year', 'lease_commence_date', 'remaining_lease']].corr(), annot=True, cmap='coolwarm')
plt.title('Heatmap: Correlation between Resale price and other variables')
plt.show()

In [ ]:
# Grouping the flats sold by month and flat model
total_units_by_month_flat_model = df.groupby(['month', 'flat_model']).size().reset_index(name='total_units')

# Convert month integers to month names
total_units_by_month_flat_model['month'] = total_units_by_month_flat_model['month'].apply(lambda x: calendar.month_abbr[x])

fig = px.line_polar(total_units_by_month_flat_model, r="total_units", theta="month", color="flat_model", line_close=True,
                    color_discrete_sequence=px.colors.sequential.Plasma_r,
                    template="plotly_dark",)
fig.show()

In [ ]:
# Grouping the flats sold by month and Year
total_units_by_month_year = df.groupby(['month', 'year']).size().reset_index(name='total_units')

# Convert month integers to month names
total_units_by_month_year['month'] = total_units_by_month_year['month'].apply(lambda x: calendar.month_abbr[x])

fig = px.line_polar(total_units_by_month_year, r="total_units", theta="month", color="year", line_close=True,
                    color_discrete_sequence=px.colors.sequential.Plasma_r
                    )
fig.show()


1.   Model Building and evaluation
2.   Linear Regression

In [28]:
# Drop irrelevant columns
selected_columns = ['year', 'town', 'flat_type', 'flat_model', 'storey_range',
                    'floor_area_sqm', 'lease_commence_date', 'remaining_lease',
                    'current_remaining_lease', 'years_holding', 'resale_price']
dftrain = df[selected_columns]

# Separate features (X) and target variable (y)
X = dftrain.drop('resale_price', axis=1)
y = dftrain['resale_price']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define preprocessing for numerical and categorical features
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# Create transformers
numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder()

# Combine transformers using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Create the regression model
model = LinearRegression()

# Create and evaluate the pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                             ('model', model)])

# Fit the model
pipeline.fit(X_train, y_train)

# save the model
model_filename = 'resale_price_prediction_linear.joblib'
joblib.dump(pipeline, model_filename)

# Predictions on the test set
y_pred = pipeline.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = sqrt(mse)
r2 = r2_score(y_test, y_pred)
explained_variance = explained_variance_score(y_test, y_pred)


# Print or use the metrics as needed
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R2) Score: {r2}")
print(f"Explained Variance Score: {explained_variance}")

Mean Absolute Error (MAE): 51686.23904678422
Mean Squared Error (MSE): 4570029691.474099
Root Mean Squared Error (RMSE): 67601.99472999372
R-squared (R2) Score: 0.835739586225564
Explained Variance Score: 0.8357404630864257


Decision trees

In [ ]:
# Drop irrelevant columns
selected_columns = ['year', 'town', 'flat_type', 'flat_model', 'storey_range',
                    'floor_area_sqm', 'lease_commence_date', 'remaining_lease',
                    'current_remaining_lease', 'years_holding', 'resale_price']
dftrain = df[selected_columns]

# Separate features (X) and target variable (y)
X = dftrain.drop('resale_price', axis=1)
y = dftrain['resale_price']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define preprocessing for numerical and categorical features
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# Create transformers
numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder()

# Combine transformers using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Create the regression model
model = DecisionTreeRegressor(random_state=42)

# Create and evaluate the pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                             ('model', model)])

# Fit the model
pipeline.fit(X_train, y_train)

# save the model
model_filename = 'resale_price_prediction_decision_tree.joblib'
joblib.dump(pipeline, model_filename)

# Predictions on the test set
y_pred = pipeline.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = sqrt(mse)
r2 = r2_score(y_test, y_pred)
explained_variance = explained_variance_score(y_test, y_pred)


# Print or use the metrics as needed
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R2) Score: {r2}")
print(f"Explained Variance Score: {explained_variance}")

In [ ]:
# Drop irrelevant columns
selected_columns = ['year', 'town', 'flat_type', 'flat_model', 'storey_range',
                    'floor_area_sqm', 'lease_commence_date', 'remaining_lease',
                    'current_remaining_lease', 'years_holding', 'resale_price']
dftrain = df[selected_columns]

# Separate features (X) and target variable (y)
X = dftrain.drop('resale_price', axis=1)
y = dftrain['resale_price']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define preprocessing for numerical and categorical features
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# Create transformers
numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder()

# Combine transformers using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Create the regression model
model = KNeighborsRegressor()

# Create and evaluate the pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                             ('model', model)])

# Fit the model
pipeline.fit(X_train, y_train)

# save the model
model_filename = 'resale_price_prediction_knn.joblib'
joblib.dump(pipeline, model_filename)

# Predictions on the test set
y_pred = pipeline.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = sqrt(mse)
r2 = r2_score(y_test, y_pred)
explained_variance = explained_variance_score(y_test, y_pred)


# Print or use the metrics as needed
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R2) Score: {r2}")
print(f"Explained Variance Score: {explained_variance}")

In [1]:
# Predictions on the test set
y_pred = pipeline.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = sqrt(mse)
r2 = r2_score(y_test, y_pred)
explained_variance = explained_variance_score(y_test, y_pred)

NameError: name 'pipeline' is not defined

Streamlit

In [ ]:
%%writefile app.py
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
import streamlit as st
import joblib

# Define unique values for select boxes
flat_model_options = ['IMPROVED', 'NEW GENERATION', 'MODEL A', 'STANDARD', 'SIMPLIFIED',
                      'MODEL A-MAISONETTE', 'APARTMENT', 'MAISONETTE', 'TERRACE', '2-ROOM',
                      'IMPROVED-MAISONETTE', 'MULTI GENERATION', 'PREMIUM APARTMENT',
                      'ADJOINED FLAT', 'PREMIUM MAISONETTE', 'MODEL A2', 'DBSS', 'TYPE S1',
                      'TYPE S2', 'PREMIUM APARTMENT LOFT', '3GEN']
flat_type_options = ['1 ROOM', '3 ROOM', '4 ROOM', '5 ROOM', '2 ROOM', 'EXECUTIVE', 'MULTI GENERATION']
town_options = ['ANG MO KIO', 'BEDOK', 'BISHAN', 'BUKIT BATOK', 'BUKIT MERAH', 'BUKIT TIMAH',
                'CENTRAL AREA', 'CHOA CHU KANG', 'CLEMENTI', 'GEYLANG', 'HOUGANG',
                'JURONG EAST', 'JURONG WEST', 'KALLANG/WHAMPOA', 'MARINE PARADE',
                'QUEENSTOWN', 'SENGKANG', 'SERANGOON', 'TAMPINES', 'TOA PAYOH', 'WOODLANDS',
                'YISHUN', 'LIM CHU KANG', 'SEMBAWANG', 'BUKIT PANJANG', 'PASIR RIS', 'PUNGGOL']
storey_range_options = ['10 TO 12', '04 TO 06', '07 TO 09', '01 TO 03', '13 TO 15', '19 TO 21',
                        '16 TO 18', '25 TO 27', '22 TO 24', '28 TO 30', '31 TO 33', '40 TO 42',
                        '37 TO 39', '34 TO 36', '46 TO 48', '43 TO 45', '49 TO 51']


# Load the saved model
model_filename = r'resale_price_prediction_decision_tree.joblib'
pipeline = joblib.load(model_filename)

# Streamlit app title
st.title("Resale Price Prediction App")

# Create a Streamlit sidebar with input fields
st.sidebar.title("Flat Details")
town = st.sidebar.selectbox("Town", options=town_options)
flat_type = st.sidebar.selectbox("Flat Type", options=flat_type_options)
flat_model = st.sidebar.selectbox("Flat Model", options=flat_model_options)
storey_range = st.sidebar.selectbox("Storey Range", options=storey_range_options)
floor_area_sqm = st.sidebar.number_input("Floor Area (sqm)", min_value=0.0, max_value=500.0, value=100.0)
current_remaining_lease = st.sidebar.number_input("Current Remaining Lease", min_value=0.0, max_value=99.0, value=20.0)
year = 2024
lease_commence_date = current_remaining_lease + year - 99
years_holding = 99 - current_remaining_lease

# Create a button to trigger the prediction
if st.sidebar.button("Predict Resale Price"):
    # Prepare input data for prediction
    input_data = pd.DataFrame({
        'town': [town],
        'flat_type': [flat_type],
        'flat_model': [flat_model],
        'storey_range': [storey_range],
        'floor_area_sqm': [floor_area_sqm],
        'current_remaining_lease': [current_remaining_lease],
        'lease_commence_date': [lease_commence_date],
        'years_holding': [years_holding],
        'remaining_lease': [current_remaining_lease],
        'year': [year]
    })

    # Make a prediction using the model
    prediction = pipeline.predict(input_data)

    # Display the prediction
    st.write("Predicted Resale Price:", prediction)

In [ ]:
# Restart Streamlit app
!streamlit run app.py &>/dev/null&

In [ ]:
!ngrok config add-authtoken 2bthDyyaiMiZYDL2k8MDnRD8GGs_6tghXCdi23Y49kGT8hSgc

In [ ]:
from pyngrok import ngrok
public_url = ngrok.connect(addr='8501')
print(public_url)